# 导入必要的库

In [1]:
import mxnet as mx
from mxnet import gluon
from mxnet import ndarray as nd
from mxnet import autograd
from mxnet.gluon import nn

import h5py
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

ctx = mx.gpu()

In [2]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

def accuracy(output, labels):
    return nd.mean(nd.argmax(output, axis=1) == labels).asscalar()

def evaluate(net, data_iter):
    loss, acc, n = 0., 0., 0.
    steps = len(data_iter)
    for data, label in data_iter:
        data, label = data.as_in_context(ctx), label.as_in_context(ctx)
        output = net(data)
        acc += accuracy(output, label)
        loss += nd.mean(softmax_cross_entropy(output, label)).asscalar()
    return loss/steps, acc/steps

# 写了一些函数

* load_data 载入之前导出的特征
* build_model 构建一个模型
* train_model 输入预训练模型的名字，进行迁移学习

In [3]:
def load_data(model_name, batch_size=128, train_size=0.8):
    features = nd.load('features_train_%s.nd' % model_name)[0]
    labels = nd.load('labels.nd')[0]

    n_train = int(features.shape[0]*train_size)

    X_train = features[:n_train]
    y_train = labels[:n_train]

    X_val = features[n_train:]
    y_val = labels[n_train:]

    dataset_train = gluon.data.ArrayDataset(X_train, y_train)
    dataset_val = gluon.data.ArrayDataset(X_val, y_val)

    data_iter_train = gluon.data.DataLoader(dataset_train, batch_size)
    data_iter_val = gluon.data.DataLoader(dataset_val, batch_size)
    
    return data_iter_train, data_iter_val

In [4]:
def build_model():
    net = nn.Sequential()
    with net.name_scope():
        net.add(nn.Dense(256, activation='relu'))
        net.add(nn.Dropout(0.5))
        net.add(nn.Dense(120))

    net.initialize(ctx=ctx)
    return net

In [5]:
def train_model(model_name):
    epochs = 50
    batch_size = 128
    
    data_iter_train, data_iter_val = load_data(model_name, batch_size)
    net = build_model()
    
    trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': 1e-4, 'wd': 1e-5})
    
    for epoch in range(epochs):
        train_loss = 0.
        train_acc = 0.
        steps = len(data_iter_train)
        for data, label in data_iter_train:
            data, label = data.as_in_context(ctx), label.as_in_context(ctx)

            with autograd.record():
                output = net(data)
                loss = softmax_cross_entropy(output, label)

            loss.backward()
            trainer.step(batch_size)

            train_loss += nd.mean(loss).asscalar()
            train_acc += accuracy(output, label)

        val_loss, val_acc = evaluate(net, data_iter_val)
    
    print("Epoch %d. loss: %.4f, acc: %.2f%%, val_loss %.4f, val_acc %.2f%% Model: %s" % (
        epoch+1, train_loss/steps, train_acc/steps*100, val_loss, val_acc*100, model_name))
    
    return val_loss

# 对所有预训练模型进行训练

In [6]:
from mxnet.gluon.model_zoo.model_store import _model_sha1

losses = []

for model_name in sorted(_model_sha1.keys()):
    val_loss = train_model(model_name)
    losses.append((model_name, val_loss))

Epoch 50. loss: 0.3581, acc: 91.84%, val_loss 1.7703, val_acc 53.16% Model: alexnet
Epoch 50. loss: 0.5784, acc: 82.65%, val_loss 0.5917, val_acc 81.61% Model: densenet121
Epoch 50. loss: 0.3868, acc: 88.30%, val_loss 0.4181, val_acc 86.60% Model: densenet161
Epoch 50. loss: 0.4281, acc: 87.34%, val_loss 0.5125, val_acc 84.01% Model: densenet169
Epoch 50. loss: 0.4015, acc: 88.35%, val_loss 0.4534, val_acc 85.09% Model: densenet201
Epoch 50. loss: 0.2417, acc: 92.57%, val_loss 0.2961, val_acc 89.73% Model: inceptionv3
Epoch 50. loss: 0.3471, acc: 89.42%, val_loss 0.4104, val_acc 87.29% Model: resnet101_v1
Epoch 50. loss: 0.3468, acc: 89.39%, val_loss 0.3994, val_acc 87.39% Model: resnet152_v1
Epoch 50. loss: 0.8433, acc: 74.35%, val_loss 0.8325, val_acc 74.19% Model: resnet18_v1
Epoch 50. loss: 0.7811, acc: 76.53%, val_loss 0.7617, val_acc 76.53% Model: resnet18_v2
Epoch 50. loss: 0.5969, acc: 82.09%, val_loss 0.5914, val_acc 82.10% Model: resnet34_v1
Epoch 50. loss: 0.5438, acc: 83.38

# 对 val_loss 进行排序

In [8]:
df = pd.DataFrame(losses, columns=['model', 'val_loss'])
df = df.sort_values('val_loss')
df.head()

,model,val_loss
5,inceptionv3,0.296050
7,resnet152_v1,0.399360
6,resnet101_v1,0.410383
2,densenet161,0.418101
4,densenet201,0.453403


In [9]:
df.to_csv('models.csv', index=None)

In [19]:
df = pd.read_csv('models.csv')

In [20]:
for i, (model_name, val_loss) in df.iterrows():
    print '%s | %s' % (model_name, val_loss)

inceptionv3 | 0.296050225385
resnet152_v1 | 0.399359531701
resnet101_v1 | 0.410383010283
densenet161 | 0.418100789189
densenet201 | 0.453403010964
resnet50_v2 | 0.484435886145
resnet50_v1 | 0.496179759502
densenet169 | 0.512498702854
resnet34_v2 | 0.536734519526
vgg19_bn | 0.557294445112
vgg16_bn | 0.586511127651
resnet34_v1 | 0.591432901099
densenet121 | 0.591716498137
vgg19 | 0.619780953974
vgg16 | 0.669267293066
vgg13_bn | 0.702507363632
vgg11_bn | 0.708396691829
vgg13 | 0.756541173905
resnet18_v2 | 0.761708110571
vgg11 | 0.789955694228
resnet18_v1 | 0.832537706941
squeezenet1.1 | 1.6066500321
squeezenet1.0 | 1.62178872526
alexnet | 1.77026221156


# 用效果最好的几个模型的特征进行训练

In [11]:
def load_models_data(model_names, batch_size=128, train_size=0.8):
    features = [nd.load('features_train_%s.nd' % model_name)[0] for model_name in model_names]
    features = nd.concat(*features, dim=1)
    labels = nd.load('labels.nd')[0]
    
    n_train = int(features.shape[0]*train_size)
    
    X_train = features[:n_train]
    y_train = labels[:n_train]

    X_val = features[n_train:]
    y_val = labels[n_train:]

    dataset_train = gluon.data.ArrayDataset(X_train, y_train)
    dataset_val = gluon.data.ArrayDataset(X_val, y_val)

    data_iter_train = gluon.data.DataLoader(dataset_train, batch_size, shuffle=True)
    data_iter_val = gluon.data.DataLoader(dataset_val, batch_size)
    
    return data_iter_train, data_iter_val

In [23]:
df.head(10)

,model,val_loss
0,inceptionv3,0.296050
1,resnet152_v1,0.399360
2,resnet101_v1,0.410383
3,densenet161,0.418101
4,densenet201,0.453403
5,resnet50_v2,0.484436
6,resnet50_v1,0.496180
7,densenet169,0.512499
8,resnet34_v2,0.536735
9,vgg19_bn,0.557294


In [51]:
net = build_model()

model_names = ['inceptionv3', 'resnet152_v1']
data_iter_train, data_iter_val = load_models_data(model_names, batch_size=batch_size)

epochs = 100
batch_size = 128
lr_sch = mx.lr_scheduler.FactorScheduler(step=400, factor=0.9)
trainer = gluon.Trainer(net.collect_params(), 'adam', 
                        {'learning_rate': 1e-4, 'wd': 1e-5, 'lr_scheduler': lr_sch})

for epoch in range(epochs):
    train_loss = 0.
    train_acc = 0.
    steps = len(data_iter_train)
    for data, label in data_iter_train:
        data, label = data.as_in_context(ctx), label.as_in_context(ctx)

        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)

        loss.backward()
        trainer.step(batch_size)

        train_loss += nd.mean(loss).asscalar()
        train_acc += accuracy(output, label)

    val_loss, val_acc = evaluate(net, data_iter_val)

    print("Epoch %d. loss: %.4f, acc: %.2f%%, val_loss %.4f, val_acc %.2f%%" % (
        epoch+1, train_loss/steps, train_acc/steps*100, val_loss, val_acc*100))

Epoch 1. loss: 4.5554, acc: 6.17%, val_loss 3.9349, val_acc 35.16%
Epoch 2. loss: 3.4425, acc: 28.50%, val_loss 2.6004, val_acc 63.87%
Epoch 3. loss: 2.3478, acc: 51.45%, val_loss 1.5700, val_acc 79.51%
Epoch 4. loss: 1.6228, acc: 65.17%, val_loss 1.0354, val_acc 84.90%
Epoch 5. loss: 1.2263, acc: 72.43%, val_loss 0.7550, val_acc 87.34%
Epoch 6. loss: 0.9907, acc: 76.36%, val_loss 0.6167, val_acc 88.07%
Epoch 7. loss: 0.8461, acc: 79.29%, val_loss 0.5285, val_acc 88.56%
Epoch 8. loss: 0.7600, acc: 80.44%, val_loss 0.4769, val_acc 89.68%
Epoch 9. loss: 0.6722, acc: 82.62%, val_loss 0.4397, val_acc 89.34%
Epoch 10. loss: 0.6256, acc: 83.62%, val_loss 0.4127, val_acc 89.49%
Epoch 11. loss: 0.5917, acc: 84.15%, val_loss 0.3908, val_acc 90.12%
Epoch 12. loss: 0.5380, acc: 85.39%, val_loss 0.3712, val_acc 89.83%
Epoch 13. loss: 0.5155, acc: 85.85%, val_loss 0.3572, val_acc 90.37%
Epoch 14. loss: 0.4787, acc: 86.82%, val_loss 0.3493, val_acc 90.12%
Epoch 15. loss: 0.4646, acc: 87.07%, val_los

# 载入测试集特征

In [52]:
features_test = [nd.load('features_test_%s.nd' % model_name)[0] for model_name in model_names]
features_test = nd.concat(*features_test, dim=1)

# 利用模型进行预测并输出到 pred.csv

In [53]:
output = nd.softmax(net(features_test.as_in_context(ctx))).asnumpy()

In [54]:
df_pred = pd.read_csv('sample_submission.csv')

for i, c in enumerate(df_pred.columns[1:]):
    df_pred[c] = output[:,i]

df_pred.to_csv('pred.csv', index=None)

# 与上次提交结果进行比对，防止错位浪费机会

In [55]:
zip(np.argmax(pd.read_csv('pred_0.28.csv').values[:,1:], axis=-1), np.argmax(df_pred.values[:,1:], axis=-1))[:10]

[(61, 61),
 (94, 94),
 (40, 40),
 (88, 88),
 (70, 70),
 (76, 76),
 (6, 6),
 (94, 94),
 (59, 59),
 (107, 107)]